In [ ]:
from labda import Subject

gps = Subject.from_parquet("../examples/test_data/gps.parquet")
# gps.set_timezone("Europe/Prague")
motus = Subject.from_parquet("../examples/test_data/motus.parquet")

motus.resample(gps.metadata.sampling_frequency, drop=True)
motus.metadata.timezone = "UTC"
motus.set_timezone(gps.metadata.timezone)
motus.id = gps.id
gps.merge(motus, how="inner")
gps.to_parquet("../examples/test_data/test_01.parquet")

In [ ]:
from labda import Subject, Features
from labda.spatial.transportation import TRANSPORTATION_CUT_POINTS

sbj = Subject.from_parquet("../examples/test_data/test_01.parquet")

params = {
    "stop_radius": 40,
    "stop_duration": "4min",
    "pause_radius": 20,
    "pause_duration": "2min",
    "min_duration": "2min",
    "indoor_limit": None,
    "turn_angle": 90,
}
sbj.detect_trips(**params, overwrite=True)
sbj.detect_transportation(TRANSPORTATION_CUT_POINTS["heidler_2025"], overwrite=True)

features = Features.from_dataframe(sbj.df, sbj.metadata.crs)
sbj.detect_locations(features, overwrite=True)

sbj.to_parquet("../examples/test_data/test_01_detected.parquet")

In [ ]:
from labda import Subject

sbj = Subject.from_parquet("../examples/test_data/test_01_detected.parquet")
# sbj.set_timeframe("2025-02-04 14:00", "2025-02-04 16:00")

sbj.plot("map", color="osm_location")

temp = sbj.df.copy()
# df = df[df["trip_id"] == 2]
trips = temp[temp["trip_status"] == "transport"]

ids = temp.groupby("trip_id")

for x, id in ids:
    statuses = id["trip_status"].unique()
    if len(statuses) > 1:
        print(x, statuses.tolist())

In [ ]:
from labda import Subject

sbj = Subject.from_parquet("../examples/test_data/test_01_detected.parquet")
summary = sbj.get_summary(["trip_transport"])
timeline = sbj.get_timeline("transport")
timeline